# Sample Sentiment Analysis

In [ ]:
!pip install nest_asyncio
from logging.handlers import TimedRotatingFileHandler
import logging.config
from psenti import SentimentAnalysis, SentimentConnection, Document
from sklearn import metrics
from datetime import datetime
import socket
import json

user_name = socket.gethostname()
host = '127.0.0.1'
port = 5000

import nest_asyncio
import asyncio
nest_asyncio.apply(loop=asyncio.get_event_loop())

logger = logging.getLogger('JupyterUI')
logFormatter = logging.Formatter('%(asctime)s - [%(thread)s] [%(threadName)s]- %(name)s - %(levelname)s - %(message)s')
logger.setLevel(logging.DEBUG)

console = logging.StreamHandler()
console.setFormatter(logFormatter)
console.setLevel(logging.DEBUG)

logger.addHandler(console)

In [ ]:
%%time

connection = SentimentConnection(host=host, port=port, client_id=user_name)
logger.info(f'Supported domains')
for domain in connection.supported_domains:    
    logger.info(f'Domain: [{domain}]')

# Standard Lexicon Sentiment Analysis

### Setup results Processing

In [ ]:
# simplified reviews
reviews = ['I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
           'I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.']

# reviews with additional metadata - date, author, id
documents = [
        Document('I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
                 '1',
                 'Ben'),
        Document('I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.',
                 '2',
                 'Ben',
                 datetime(1995, 5, 2))
        ]

def process_result(domain, result):
    if result['Stars'] is None:
        logger.warning(f'{domain}: [{result["Id"]}] No Sentinent')
    else:
        logger.info(f'{domain}: [{result["Id"]}] Sentinment Stars: {result["Stars"]:1.2f}')     
        
    for sentence in result['Sentences']:
        for word in sentence['Words']:
            if 'Value' in word:
                logger.info(f"[{word['Span']}] Sentiment: {word['Value']:1.2f}")     


### Analyse using standard sentiment lexicon

In [ ]:
analysis = SentimentAnalysis(connection)
analysis.on_message.subscribe(lambda result: process_result(domain, result))
analysis.detect_sentiment_text(reviews)

### Analyse using custom domain lexicons

In [ ]:
for domain in connection.supported_domains:
    # analyse using custom domain lexicons
    analysis = SentimentAnalysis(connection, domain=domain)
    analysis.on_message.subscribe(lambda result: process_result(domain, result))
    analysis.detect_sentiment(documents)

### Analysis Using Custom lexicon

In [ ]:
lexicon = {'love': -3, 'bad oder': 3, 'bad': -1} # only this lexicon will be in use
analysis = SentimentAnalysis(connection, lexicon=lexicon)
analysis.on_message.subscribe(lambda result: process_result(domain, result))
analysis.detect_sentiment_text(reviews)

### Modification of existing lexicon

In [ ]:
lexicon = {'love': -3, 'bad oder': 3} # lexicon modifaction

# without modification
analysis = SentimentAnalysis(connection, domain='medical')
analysis.on_message.subscribe(lambda result: process_result('Medical', result))
analysis.detect_sentiment(documents)

# with modification
analysis = SentimentAnalysis(connection, domain='medical', lexicon=lexicon, adjust_lexicon=True)
analysis.on_message.subscribe(lambda result: process_result('Adjusted Medical', result))
analysis.detect_sentiment(documents)